# Práctica integral: Bases de Datos en Databricks (Lakehouse)
**Duración estimada:** ~2 horas  
**Tono:** Académico y formal  
**Entorno objetivo:** Databricks Premium Trial (AWS)  
**Ruta base (Volumen de trabajo):** `/Volumes/jengrik_dbx_1576397358716270/default/jengrik-volume/`

> Este cuaderno combina teoría y práctica para construir una base de datos de complejidad media (clientes, productos, pedidos y renglones),
> habilitando consultas SQL, operaciones DML, _Time Travel_ de Delta y visualizaciones con Python.


## 1. Requisitos previos y consideraciones de entorno

### Objetivo de la sección 1
Comprender las condiciones técnicas necesarias para ejecutar correctamente los ejercicios prácticos en Databricks Premium Trial sobre AWS.

### Qué se realizará en la sección 1
Se verificará el entorno, el clúster activo, las rutas de almacenamiento (Volumes) y las herramientas disponibles. Se introducirá el uso de celdas SQL y Python en un mismo notebook.

### Por qué es importante hacerlo (sección 1)
Garantizar un entorno controlado es esencial para la reproducibilidad de resultados y la gestión de recursos en proyectos de ingeniería de datos distribuidos.


**Requisitos del entorno**  
- **Cluster** activo (Runtime Spark 13.x+ recomendado).  
- **Unity Catalog** y **Volumes** disponibles. Usaremos el volumen base: `/Volumes/jengrik_dbx_1576397358716270/default/jengrik-volume/` para persistir archivos “raw” (CSV/Parquet) y, opcionalmente, tablas Delta externas.  
- Permisos adecuados de lectura/escritura sobre el volumen.

**Nota sobre Databricks SQL/Notebooks**  
- En celdas de este notebook se muestran fragmentos con magia `%sql` para ejecutar **SQL** directamente.
- Las celdas **Python** usarán PySpark y APIs de Delta cuando aplique.
- Si ejecutas fuera de Databricks, ignora las celdas `%sql` y replica la lógica desde Python.

### Cosas para recordar
- Un **Volume** es un espacio de archivos gobernado por Unity Catalog.
- Un **Schema (Database)** es un contenedor lógico de tablas/vistas.
- Puedes tener **tablas manejadas** o **tablas externas**.

### Importancia en Ingeniería de Datos
- Distinguir almacenamiento **lógico** del **físico** es clave para la gobernanza y la reproducibilidad.
- Los Volumes facilitan el patrón **medallion** (raw/bronze, silver, gold).

### Conclusiones
- Preparar el entorno y permisos reduce fricción técnica durante la práctica.


## 2. Bases de datos lógicas vs. almacenamiento físico

### Objetivo de la sección 2
Distinguir los conceptos de base de datos, esquema, tabla lógica y almacenamiento físico dentro del modelo Lakehouse de Databricks.

### Qué se realizará en la sección 2
Se analizará cómo las bases de datos y tablas lógicas en el catálogo se relacionan con los archivos reales ubicados en Volumes o S3.

### Por qué es importante hacerlo (sección 2)
Comprender esta relación permite diseñar arquitecturas gobernadas, donde los metadatos y los datos pueden evolucionar de forma independiente.


**Bases de datos lógicas (catálogo/esquema):**  
- Estructuran y gobiernan **metadatos** (nombres, columnas, permisos, lineage).  
- Separan la **vista lógica** del **lugar físico** donde residen los datos.

**Almacenamiento físico (Volumes / S3 / External Locations):**  
- Contiene los **archivos reales**: Delta, Parquet, CSV, etc.  
- La misma tabla lógica puede apuntar a diferentes ubicaciones físicas a lo largo del tiempo.

**Tablas manejadas vs. externas:**  
- **Manejadas**: El motor controla el _path_ y ciclo de vida de los archivos.  
- **Externas**: El metastore referencia un _path_ explícito. Tú gestionas el ciclo de vida de los archivos.

### Cosas para recordar
- No toda tabla Delta es manejada; depende de cómo fue creada.  
- El **catálogo** registra lo que hay; los **archivos** son la verdad última de los datos.

### Importancia en Ingeniería de Datos
- Desacopla la evolución del esquema lógico del repositorio físico.
- Habilita **gobernanza** (ACLs, lineage) sin perder flexibilidad operativa.

### Conclusiones
- El Lakehouse depende de una relación clara entre **metadatos** (catálogo) y **datos** (storage).


## 3. Creación de esquema y preparación del entorno

### Objetivo de la sección 3
Aprender a crear y utilizar esquemas (bases de datos) en Databricks para organizar objetos lógicos y físicos.

### Qué se realizará en la sección 3
Se usará el comando `CREATE SCHEMA` y se definirá el esquema activo mediante `USE`. También se parametrizará la ruta base del volumen de trabajo.

### Por qué es importante hacerlo (sección 3)
Definir correctamente el esquema garantiza aislamiento entre ambientes y facilita la trazabilidad de los objetos creados.


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS demo_db_jengrik;
USE demo_db_jengrik;

-- Verificar el esquema actual
SELECT current_catalog(), current_schema()



In [0]:
# Python: variables base (útil para parametrizar rutas)



### Cosas para recordar
- `CREATE SCHEMA` crea un contenedor lógico.  
- `USE <schema>` cambia el contexto de ejecución SQL.

### Importancia en Ingeniería de Datos
- Separar ambientes y dominios de negocio por esquemas facilita el **aislamiento** y la **gestión de permisos**.

### Conclusiones
- Un esquema dedicado a la práctica evita colisiones y mejora la trazabilidad.


## 4. Generación de datos sintéticos y creación de tablas Delta

### Objetivo de la sección 4
Construir un conjunto de datos de tamaño mediano para simular una base de datos empresarial.

### Qué se realizará en la sección 4
Se generarán datos de clientes, productos, pedidos y renglones, se guardarán en formato CSV/Parquet (zona raw) y luego se crearán tablas Delta manejadas.

### Por qué es importante hacerlo (sección 4)
Esta práctica permite comprender el flujo de ingestión y consolidación, así como el rol de Delta Lake en la confiabilidad de los datos.


In [0]:
# Python: Generar datos sintéticos 'medianos' y persistir archivos raw (CSV/Parquet) en el Volume.
# Luego, crear tablas Delta manejadas a partir de esos datos.



### Cosas para recordar
- Separar **RAW (archivos)** de **tablas Delta** facilita el linaje y auditoría.
- `saveAsTable` crea tablas **manejadas** por el metastore.

### Importancia en Ingeniería de Datos
- Disponer de datos “medianos” habilita **agregaciones** y **gráficos** con significado.
- CSV y Parquet como formatos de **ingesta**; Delta como formato de **consumo**.

### Conclusiones
- Partir de archivos RAW y consolidar en Delta es un patrón común (Bronze → Silver).


## 5. Consultas SQL y agregaciones analíticas

### Objetivo de la sección 5
Aplicar consultas SQL sobre tablas Delta para obtener información de valor analítico.

### Qué se realizará en la sección 5
Se ejecutarán consultas de exploración, uniones entre tablas de hechos y dimensiones, y agregaciones como ventas por ciudad o categoría.

### Por qué es importante hacerlo (sección 5)
Ejercitar el lenguaje SQL dentro de Databricks refuerza la integración entre análisis y almacenamiento distribuido.


In [0]:
# Databricks: ejecutar como SQL
%sql


### Cosas para recordar
- Mantén **claves sustitutas** simples (IDs enteros) para joins eficientes.
- Prefiere agregaciones sobre **order_items** para granularidad de ventas.

### Importancia en Ingeniería de Datos
- Las uniones entre dimensiones y hechos son el corazón del **modelo analítico**.
- Diseños claros reducen costos de computación y errores de calidad.

### Conclusiones
- Con un modelo sencillo ya podemos responder preguntas de negocio relevantes.


## 6. Operaciones DML e introducción a vistas temporales

### Objetivo de la sección 6
Explorar las operaciones de manipulación de datos (INSERT, UPDATE) y la creación de vistas temporales para análisis ad-hoc.

### Qué se realizará en la sección 6
Se insertarán nuevos registros, se modificarán valores existentes y se definirán vistas temporales basadas en criterios de negocio.

### Por qué es importante hacerlo (sección 6)
Estas operaciones permiten validar hipótesis analíticas y mantener entornos de prueba controlados.


In [0]:
# Databricks: ejecutar como SQL
%sql


### Cosas para recordar
- `INSERT` y `UPDATE` son soportados por Delta Lake.
- Las **vistas temporales** viven dentro de la sesión del cluster.

### Importancia en Ingeniería de Datos
- Las operaciones DML controladas ayudan a **simular escenarios** y validar reglas de negocio.
- Las vistas permiten **encapsular** lógicas de análisis sin materializar tablas.

### Conclusiones
- El flujo _ingesta → DML → vistas_ es una base sólida para análisis y prototipos.


## 7. Versionado y Time Travel en Delta Lake

### Objetivo de la sección 7
Comprender cómo Delta Lake registra automáticamente versiones de cada escritura y cómo acceder a ellas.

### Qué se realizará en la sección 7
Se examinará el historial de una tabla Delta con `DESCRIBE HISTORY` y se realizarán consultas a versiones anteriores.

### Por qué es importante hacerlo (sección 7)
El Time Travel es esencial para auditoría, depuración y reproducibilidad de resultados.


In [0]:
# Databricks: ejecutar como SQL
%sql



### Cosas para recordar
- Cada escritura en una tabla Delta crea una **versión**.
- `DESCRIBE HISTORY` revela operaciones, _user_, _timestamp_, _operationMetrics_.

### Importancia en Ingeniería de Datos
- **Auditoría** y **reproducibilidad**: volver a un estado anterior para depuraciones o comparativas históricas.

### Conclusiones
- _Time Travel_ fortalece la **confiabilidad** y acelera el **debugging** en pipelines.


## 8. Visualización y análisis gráfico con Python

### Objetivo de la sección 8
Integrar consultas SQL con visualizaciones en Python para interpretar resultados de manera visual y comprensible.

### Qué se realizará en la sección 8
Se utilizará matplotlib para graficar ventas diarias, por ciudad y por categoría a partir de los datos generados.

### Por qué es importante hacerlo (sección 8)
La visualización de datos es clave para detectar patrones, anomalías y validar supuestos analíticos.


In [0]:
# Python: Visualizaciones básicas (usar matplotlib, sin estilos ni subplots).



### Cosas para recordar
- Las visualizaciones deben responder preguntas concretas.
- Evita gráficos redundantes; prioriza claridad y legibilidad.

### Importancia en Ingeniería de Datos
- Validaciones visuales de tendencias y outliers.
- Complementa el SQL con exploración visual.

### Conclusiones
- Series temporales y barras por dimensión aportan valor inmediato.


## 9. Creación de tablas Delta externas

### Objetivo de la sección 9
Aprender a registrar tablas Delta que apuntan a rutas explícitas dentro de Volumes.

### Qué se realizará en la sección 9
Se materializarán tablas Delta externas a partir de las existentes y se registrarán con `USING DELTA LOCATION`.

### Por qué es importante hacerlo (sección 9)
Las tablas externas son útiles para interoperabilidad o cuando la organización gestiona el almacenamiento físico.


In [0]:
# Databricks: ejecutar como SQL
%sql



In [0]:
# Python: materializar Delta externo en Volume y registrar tabla externa.



In [0]:
# Databricks: ejecutar como SQL
%sql



### Cosas para recordar
- `USING DELTA LOCATION` registra tablas **externas** referenciando rutas explícitas.
- La **vida** de los datos está desacoplada de la vida del objeto lógico en el metastore.

### Importancia en Ingeniería de Datos
- Útil para interoperar con otras herramientas o cuando el storage se gestiona fuera de Databricks.

### Conclusiones
- Tablas externas aportan flexibilidad; requieren disciplina en housekeeping.


## 10. Reflexiones adicionales (evaluación formativa)

### Objetivo de la sección 10
Consolidar los aprendizajes conceptuales y prácticos mediante preguntas de análisis y reflexión.

### Qué se realizará en la sección 10
Se propondrán ejercicios orientados a evaluar decisiones arquitectónicas, calidad de datos y aplicación de Time Travel.

### Por qué es importante hacerlo (sección 10)
La evaluación formativa refuerza la comprensión crítica de fundamentos de ingeniería de datos y su aplicación práctica.


1. ¿En qué escenarios conviene preferir **tablas manejadas** frente a **externas**? Justifique con criterios de gobierno y operación.
2. Diseñe un **índice de calidad de datos** (simple) para `orders` y proponga validaciones mínimas (nulos, rangos, duplicados).
3. Elabore un pequeño **experimento** de _Time Travel_ que muestre la trazabilidad de un cambio de precio en `products`.
4. Pregunta abierta: ¿Qué **métricas de negocio** adicionales calcularía (margen, ticket promedio, recurrencia de compra) y cómo validaría su consistencia?

### Cosas para recordar
- La calidad de datos y la trazabilidad impactan directamente la **confiabilidad** del análisis.
- Las decisiones de modelado y gobierno deben alinearse con el **dominio de negocio**.

### Importancia en Ingeniería de Datos
- La evaluación formativa promueve pensamiento crítico sobre **trade-offs** arquitectónicos.
- Refuerza buenas prácticas de **gobernanza** y **observabilidad**.

### Conclusiones
- Más allá del código, el valor proviene de interpretaciones sólidas y decisiones informadas.


## 11. Cierre y proyección profesional

### Objetivo de la sección 11
Sintetizar los conocimientos adquiridos sobre bases de datos, Delta Lake y buenas prácticas en Databricks.

### Qué se realizará en la sección 11
Se revisarán los pasos ejecutados, se identificarán oportunidades de mejora y se propondrán líneas de trabajo futuras.

### Por qué es importante hacerlo (sección 11)
El cierre permite consolidar aprendizajes y conectar la práctica con procesos reales de automatización y gobernanza de datos.


Hemos construido una base de datos de tamaño mediano con un modelo lógico claro y datos suficientes para análisis y visualización.
Se cubrieron: creación de esquema, ingesta de datos, consolidación en Delta, consultas SQL, DML, Time Travel, tablas externas y gráficos.

**Siguiente paso sugerido:** Integrar este cuaderno a un **repositorio Git** y preparar un **job** programado para refrescar datos y generar reportes periódicos.
